<a href="https://colab.research.google.com/github/kochol/tf_tests/blob/master/test01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test 01

This is the first test of my tf code.

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

# enable tf eager execution to support iterating
tf.enable_eager_execution()

print(tf.VERSION)
print(tf.keras.__version__)

1.13.1
2.2.4-tf


## Create the model

Now create the model

In [0]:
inputs = tf.keras.Input(shape=(8,))  # Returns a placeholder tensor

# A layer instance is callable on a tensor, and returns a tensor.
x = layers.Dense(64, activation='relu')(inputs)
x = layers.Dense(64, activation='relu')(x)
predictions = layers.Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs=inputs, outputs=predictions)

# I want to calc on float numbers so I use mse and mae for loss and metrics
# Also I have to find out which optimizer is good for me.
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='mse',
              metrics=['mae'])

## Load data from csv file

In [6]:
# load all data
dataset_x = tf.data.experimental.CsvDataset(
    filenames='mydata.csv',
    record_defaults=[tf.int64, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.float32, tf.int32],
    header=True)

print(dataset_x.output_classes)

# load only close col
dataset_y = tf.data.experimental.CsvDataset(
    filenames='mydata.csv',
    record_defaults=[tf.float32],
    select_cols=[4],
    header=True)

print(dataset_y.output_classes)


(<class 'tensorflow.python.framework.ops.Tensor'>, <class 'tensorflow.python.framework.ops.Tensor'>, <class 'tensorflow.python.framework.ops.Tensor'>, <class 'tensorflow.python.framework.ops.Tensor'>, <class 'tensorflow.python.framework.ops.Tensor'>, <class 'tensorflow.python.framework.ops.Tensor'>, <class 'tensorflow.python.framework.ops.Tensor'>, <class 'tensorflow.python.framework.ops.Tensor'>)
(<class 'tensorflow.python.framework.ops.Tensor'>,)


## Train the model

In [8]:
dataset_x = dataset_x.batch(1).repeat()
dataset_y = dataset_y.batch(1).repeat()

model.fit(dataset_x, epochs=10, steps_per_epoch=30)

Epoch 1/10


ValueError: ignored